In [2]:
from InSituToolkit.imaging_database import write_experiment
import imaging_db.database.db_operations as db_ops
import imaging_db.filestorage.s3_storage as s3_storage
import imaging_db.filestorage.local_storage as local_storage
import imaging_db.utils.db_utils as db_utils
import os, csv, pickle

%gui qt5
import numpy as np
from skimage import io
from starfish import Experiment, display, Codebook, ExpressionMatrix, FieldOfView, BinaryMaskCollection, LabelImage
from starfish.image import Filter
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.types import Axes, Coordinates, Features, FunctionSource, TraceBuildingStrategies

from InSituToolkit.analysis import save_stack

db_credentials = '/Users/andrew.cote/Documents/db_credentials.json'
root_path = '/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/'
# TODO: something clever with loading the experiment names, determining the assay, and dataset ID to get all the positions

# load the list of experiments and iterate over all of them TODO
# list_of_experiments = pickle.load(open('list_of_experiments.p'))
# for path in list_of_experiments[0:1]:

'''
this particular experiment has 12 FOVs, 
RNAscope assay #2:
RNAscope staining 4. NKX2-1: C2 Opal dye 620
RNAscope staining 5. SELENBP1: C3 Opal dye 690
RNAscope staining 6. IGFBP3: C1 Opal dye 570
''' 

#TODO: setup a small script that loads up the binary mask as output from ilastik into napari along with some fovs and
# send over to Ashley for her to look over. 

'''
Plan of attack:
- create a pipeline in Ilastik that can classify regions as stroma or non-stroma
- Produce a binary mask where non-stroma regions are '1' and stroma regions are '0'
- Logical AND this mask with all three channels on the fovs such that we are only left with non-zero intensities
    in non-stroma regions
- find spots using bandpass filter and blob-detector
- construct codebook according to the type of assay used (will need to read off file-names etc)
- produce the count x cell matrix, where there is only 'one' cell which is non-stroma region
- estimate total area of non-stroma region in pixels (could just sum elements of binary mask matrix)
- produce density of target genes, i.e. number of spots/pixel area, for each channel

Some notes:
- binary mask will be the same across all channels
- Check in with Ashley about whether the stroma and non-stroma classification is valid
- could either use the binary mask to force pixel intensities to be zero, or just use it as a label image for the 
    codebook gene assignment
'''

"\nPlan of attack:\n- create a pipeline in Ilastik that can classify regions as stroma or non-stroma\n- Produce a binary mask where non-stroma regions are '1' and stroma regions are '0'\n- Logical AND this mask with all three channels on the fovs such that we are only left with non-zero intensities\n    in non-stroma regions\n- find spots using bandpass filter and blob-detector\n- construct codebook according to the type of assay used (will need to read off file-names etc)\n- produce the count x cell matrix, where there is only 'one' cell which is non-stroma region\n- estimate total area of non-stroma region in pixels (could just sum elements of binary mask matrix)\n- produce density of target genes, i.e. number of spots/pixel area, for each channel\n\nSome notes:\n- binary mask will be the same across all channels\n- Check in with Ashley about whether the stroma and non-stroma classification is valid\n- could either use the binary mask to force pixel intensities to be zero, or just us

In [3]:
list_of_datasets = pickle.load(open('list_of_experiments.obj', 'rb'))
dict_of_datasets = pickle.load(open('dict_of_experiments.obj', 'rb'))
CODEBOOK = pickle.load(open('codebook.obj', 'rb'))

In [ ]:
for dataset in list_of_datasets:
    save_path, exp_name, assayNo = dict_of_datasets[dataset]

    exp = Experiment.from_json(save_path + 'experiment.json')
    exp_name_safe = exp_name.replace('/', '-')

    # get all fovs from the experiment
    fovs = [k for k in exp.keys()]

    # To import images into Ilastik, we want to take the max projection of z-Planes across all color channels, 
    # since the stroma tissue flouresces brightly and we want to paint these regions in Ilastik. 
    for fov in fovs:
        img_stack = next(exp[fov].get_images(FieldOfView.PRIMARY_IMAGES))
        img_stack_sel = img_stack.sel({Axes.CH: 0})
        img_stack_reduced = img_stack_sel.reduce({Axes.ZPLANE}, func='max')
        save_stack(img_stack_reduced, root_path + 'ilastik/reduced_fovs/' + exp_name_safe + fov + '.tif')

100%|██████████| 1/1 [00:00<00:00, 89.68it/s]
/opt/anaconda3/envs/insitu/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/opt/anaconda3/envs/insitu/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: /Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/ilastik/reduced_fovs/AZ017_SURG_assay2-roi3-fov_000.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


In [ ]:
'''
At this point we have exported all the reduced / projected images and then classified them using Ilastik
'''

# assume we are working with the fov_000 from dataset[0]
fov = 'fov_000'
dataset = list_of_datasets[0]

# the rest below should be iterated over for each fov
save_path, exp_name, assayNo = dict_of_datasets[dataset]
codebook = CODEBOOK[int(assayNo)]
exp = Experiment.from_json(save_path + 'experiment.json')

# now we must import the segmented ilastik images
classified_img_path = root_path + 'ilastik/classified_fovs/' + exp_name + fov + '.tif'
label_image = io.imread(classified_img_path)

# we must also get the physical coords from the original to produce the binary mask

yc = img_stack.xarray.yc.values
xc = img_stack.xarray.xc.values
physical_ticks = {Coordinates.Y: yc, Coordinates.X:xc}

y = img_stack.xarray.y.values
x = img_stack.xarray.x.values
pixel_coords = {Axes.Y: y, Axes.X: x}

label_im = LabelImage.from_array_and_coords(label_image, \
                                            pixel_coordinates=pixel_coords, \
                                            physical_coordinates=physical_ticks, \
                                           log = img_stack.log)

In [ ]:
# we are basically doing edge detect on gaussian spatial features
# this is done once per fov

img_stack = next(exp[fov].get_images(FieldOfView.PRIMARY_IMAGES))

ghp = Filter.GaussianHighPass(sigma=3)
high_passed = ghp.run(img_stack, verbose=True, in_place=False)

glp = Filter.GaussianLowPass(sigma=1)
low_passed = ghp.run(high_passed, verbose=True, in_place=False)

In [ ]:
low_passed.reduce

In [ ]:
d = {2: 'cat'}
d[2]

In [ ]:
# Save the GFP stack

gfp = exp.fov().get_image('stain')
gfp_mip = mproj.run(gfp)
save_stack(gfp_mip, './find_spots/gfp.tif')

In [ ]:
#better to use the FieldOfView.ENUM designator as the string matching to 'primary may change later'

primary_000 = exp['fov_001'].get_images(FieldOfView.PRIMARY_IMAGES)
type(primary_000)

In [ ]:
image_stack = next(primary_000)
viewer = display(image_stack)
display()

In [ ]:
# project z planes by maximum

mproj = Filter.Reduce((Axes.ZPLANE,), func='max', module=FunctionSource.np)
mip = mproj.run(low_passed)

In [ ]:
# find the spots 

p = FindSpots.BlobDetector(
    min_sigma = 1, max_sigma = 10, num_sigma = 10, threshold = 0.001, 
    measurement_type = 'mean'
)

In [ ]:
# To view a collection of FOVs in napari:
img_stack = next(exp[fovs[0]].get_images(FieldOfView.PRIMARY_IMAGES))
viewer = display(img_stack)

for fov in fovs[0:4]:
    display(img_stack, viewer=viewer)